Этот ноутбук создан локально на Mac M4, для настройки окружения

brew install python@3.11    
/opt/homebrew/bin/python3.11 -m venv vllm_env    
source vllm_env/bin/activate    
pip install --upgrade pip setuptools wheel     
brew install cmake pkg-config rust    
pip install -q numpy    
pip install -q sentencepiece --prefer-binary    
pip install sentencepiece==0.1.99    
pip install vllm     

Если используете Jupyter

pip -q install ipykernel     
python -m ipykernel install --user --name=vllm_env --display-name "Python 3.11 (vllm_env)"    

In [1]:
!CMAKE_ARGS="-DLLAMA_METAL=on" pip install llama-cpp-python --force-reinstall --no-cache-dir -q

In [2]:
!mkdir -p models/mistral

In [3]:
%%capture
!wget  -O models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf \
  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf

In [5]:
!ls models/mistral/

mistral-7b-instruct-v0.1.Q2_K        mistral-7b-instruct-v0.1.Q8_0.gguf
mistral-7b-instruct-v0.1.Q4_K_M.gguf


В терминале выполняем

python3 -m llama_cpp.server --model ./models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf --n_gpu_layers 35 --n_ctx 2048

Проверим что модель действительно доступна

In [1]:
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"./Downloads/models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf","object":"model","owned_by":"me","permissions":[]}]}

In [4]:
# %pip -q install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip -q install langchain langchain_community

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -Uq langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -q pypdf

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install -q sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install -q faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [8]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("https://ussr2.ru/_ld/0/79_Uells_G-Rossia_.pdf")
documents = loader.load_and_split()
len(documents)

221

In [9]:
from langchain.text_splitter import CharacterTextSplitter
splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False)
documents = loader.load_and_split(splitter)
len(documents)

453

In [10]:
for doc in documents:
    doc.page_content = doc.page_content.replace('-\n', "").replace('\n', "")

In [11]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

/var/folders/dd/3y87lw9x15l1vkmh5jvkhcc80000gn/T/ipykernel_61178/947060026.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
/Users/nikita/llama_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
db = FAISS.from_documents(documents, embeddings)

In [31]:
retriever = db.as_retriever(
    search_type="similarity",
    k=3,
    score_threshold=None,
)

In [27]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="local-mistral",  # как больше нравится
    openai_api_base="http://localhost:8000/v1", 
    openai_api_key="xxxx",  # любое значение, проверка не идёт
    temperature=0.7,
)

In [34]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

In [35]:
query = "Что увидел Герберт Уэллс в России?"
result = qa.invoke(query)

print("Ответ:", result["result"])

Ответ:  Based on the provided context, Herbert Uelsmann visited Russia in 1920 and saw a country that was in ruins after the Russian Revolution. He was particularly struck by the lack of energy and resources in Russia, which made it seem like the country would never be able to recover from its current state. However, he was also impressed by Vladimir Lenin's vision for the future of Russia and his plans for electrification, which Uelsmann described as "forward-looking." He named Lenin "the dreamer" based on this conversation.


Попробуем кастомный промпт

In [18]:
from langchain.prompts import PromptTemplate
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
        Ты — историк, внимательный читатель. Используй приведённый ниже контекст, чтобы ответить на вопрос. 
        Если ответа нет в контексте, честно скажи "я не знаю".
        
        Контекст:
        {context}
        
        Вопрос:
        {question}
        
        Ответ:
        """.strip()
        )

# Создаём RetrievalQA с кастомным промптом
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": custom_prompt}
)

query = "Что увидел Герберт Уэллс в России?"
result = qa_chain.invoke(query)

print("Ответ:", result["result"])

Ответ:  Герберт Уэллс в России видел разруху, особенно в Петербурге. Он знал, что без энергичной помощи извне, Россия не сможет выйти из бедности, в которой упала. Поэтому, когда Ленин в такой обстановке начал развивать план электрификации перед ним, Уэллс назвал его «кремлевским мечтателем».


In [19]:
query = "Кем работал Герберт Уэллс в юности?"
result = qa_chain.invoke(query)

print("Ответ:", result["result"])

Ответ:  Используя контекст, можно предположить что в юности Герберт Уэллс работал в мануфактурную лавку.


In [20]:
query = "Кем мать устроила работать Герберта Уэллса в юности?"
result = qa_chain.invoke(query)

print("Ответ:", result["result"])

Ответ:  я не знаю


In [21]:
query = "Куда мать устроила работать Герберта Уэллса в юности?"
result = qa_chain.invoke(query)

print("Ответ:", result["result"])

Ответ:  Я не знаю.


In [23]:
retriever.get_relevant_documents(query)[0].page_content

'щегольски  одетым  и,  увы,  каким  буржуа!»  1  — рассказывала  и 1946 году  в  «Стейгсмен  энд  нейшн»  Дженни  Хорстин,  которая в 1920  году  петроградской  девочкой  Женей  Лунц  видела  Уэллса во время  посещения  им  тенишевской  гимназии.  «Я  выругал этого Уэллса  с  наслаждением  в  Доме  искусств,—  вспоминал В. Шкловский.—  Алексей  Максимович  радостно  сказал  переводчице:  «Вы  это  ему  хорошо  переведите»  2.  Он  казался  слишком благополучным  на  фоне  всеобщей  нищеты,  слишком  рассудительным  в  одних  случаях,  слишком  придирчивым  в  других. Он был  эмоционально  неприемлем  для  людей,  привыкших  к тяготам  этих  лет  и  видевших  в  них  свою  дань  будущему.  Мало даже  сказать,  что  Уэллс  не  умел  слушать  музыку  революции,— он просто  не  слышал  ее. И вместе  с  т,ем.  «Россия во  мгле»  — честная  книга. Чтобы  увидеть  и  рассказать  то,  что  увидел  и  рассказал Уэллс, не  требовалось  быть  сторонником  большевистской  пар-'